# Conversational Interface - Chatbot with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers.Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.


## Chatbot using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


### Use Cases

1. **Chatbot (Basic)** - Zero Shot chatbot with a FM model
2. **Chatbot using prompt** - template(Langchain) - Chatbot with some context provided in the prompt template
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions

#### Langchain framework for building Chatbot with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## DynamoDB Table for Chat History Storage
Before we start to understand how Bedrock and LangChain can be used to create a conversational interface, we need to create a table using Amazon DynamoDB which will be used to persist the chat history between you and the LLM.

In [ ]:
import boto3

# Get the service resource.
dynamodb = boto3.resource("dynamodb")

try:
    # Create the DynamoDB table.
    table = dynamodb.create_table(
        TableName="ChatSessionTable",
        KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
    # Wait until the table exists.
    table.meta.client.get_waiter("table_exists").wait(TableName="ChatSessionTable")

except Exception as e:
        if e.__class__.__name__ == 'ResourceInUseException':
            table = dynamodb.Table("ChatSessionTable")
        else:
             raise e
# Print out some data about the table.
print(table.item_count)

## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory

modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
history = DynamoDBChatMessageHistory(table_name="ChatSessionTable", session_id="1")
memory = ConversationBufferMemory(
    memory_key="history", chat_memory=history, return_messages=True
)
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.run(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

To learn more about how to write prompts for Claude, check [Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

## Chatbot using prompt template (Langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

history = DynamoDBChatMessageHistory(table_name="ChatSessionTable", session_id="2")
memory = ConversationBufferMemory(
    memory_key="history", chat_memory=history, return_messages=True
)

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant: """)

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="Hi there!"))

#### New Questions

Model has responded with intial message, let's ask few questions

In [ ]:
print_ww(conversation.run(input="Give me a few tips on how to start a new garden."))

#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [ ]:
print_ww(conversation.run(input="Cool. Will that work with tomatoes?"))

#### Finishing this conversation

In [ ]:
print_ww(conversation.run(input="That's all, thank you!"))

Claude is still really talkative. Try changing the prompt to make Claude provide shorter answers.

## Chatbot with persona

AI assistant will play the role of a career coach. Role Play Dialogue requires user message to be set in before starting the chat. ConversationBufferMemory is used to pre-populate the dialog

In [ ]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
history = DynamoDBChatMessageHistory(table_name="ChatSessionTable", session_id="3")
memory = ConversationBufferMemory(
    memory_key="history", chat_memory=history, return_messages=True
)
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

print_ww(conversation.run(input="What are the career options in AI?"))

In [ ]:
print_ww(conversation.run(input="What these people really do? Is it fun?"))

##### Let's ask a question that is not specialty of this Persona and the model shouldn't answer that question and give a reason for that

In [ ]:
conversation.verbose = False
print_ww(conversation.run(input="How to fix my car?"))

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens_to_sample` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas
